# Chapter 22

# Working with non-Normal distributions and the generalized linear model


Up to now, we have only worked with **numeric variables**. However, **categorical variables** are extremely common. In fact, most real-world data science problems involve one or more categorical variables. Indeed, some types of analysis involve only categorical variables. For examine, in natural language processing, all variables are categorical in the form of words or phrases. 

Machine learning models can have categorical variables as features. But, most models require numeric predictor variables. The question is, how can we transform categorical variables into numeric variables?  

In this notebook you will be introduced to working with categorical variables as features in regression models. The methods explored apply to most any machine learning model.  

## Preparing the Data Set

In this section we will prepare the data set we will use in this notebook. As a first step, execute the code in the cell below to import the required packages.

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
import statsmodels.api as sm
import statsmodels.formula.api as smf  
from statsmodels.graphics.regressionplots import influence_plot, plot_regress_exog
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize, StandardScaler
from patsy import dmatrices
from sklearn import metrics
import contextlib
import io

%matplotlib inline
sns.set(style='ticks', palette='Set2')

Execute the cell below to perform load the dataset into a data frame and examine the data types of the columns. 

In [ ]:
auto_data = pd.read_csv('../data/AutoPricesClean.csv')
auto_data.dtypes                      

There are a number of numeric variables and categorical variables, of type `object`.

Next, execute the code in the cell below to do the following:   

- Remove the first 3 columns of the data frame. 
- Standardize the numeric columns of the data frame, except for the last 3 we can use as labels.  
- Examine the head of the data frame.   

In [ ]:
## Get a list of columns that are not of type object   
auto_data.drop(auto_data.columns[:3], axis=1, inplace=True)
numeric_columns = [col for col_type,col in zip(auto_data.iloc[:,:-3].dtypes,auto_data.iloc[:,:-3].columns) if col_type in ['int64','float64']]
auto_data.loc[:,numeric_columns] = StandardScaler().fit_transform(auto_data.loc[:,numeric_columns])
auto_data.head()

Our goal with these data is to model the city MPG. Therefore, we will start with displaying a graph of the city MPG vs. curb weight for the two fuel types. Execute the code in the cell below and examine the  

In [ ]:
def plot_auto(x='curb_weight', y='city_mpg', hue='fuel_type', 
              title = 'City MPG by curb weight and fuel type', 
              xlabel='Curb weight', ylabel='City MPG',
              data=auto_data):
    fig,ax = plt.subplots(figsize=(8,8))
    sns.scatterplot(x=x, y=y, hue=hue, data=data);
    ax.set_title(title);
    ax.set_xlabel(xlabel);
    ax.set_ylabel(ylabel);
    return ax
    
_=plot_auto()    

## Categorical Variables and the Model Matrix

Most machine learning models can only work with numeric variables. Therefore, ee need to encode the categorical variables into one or more numeric variables. The common approach is to convert the categorical variable to a set of binary **dummy variables** or **indicator variables**. We call this process **one-hot encoding** since only one of the dummy variables will be encoded as a 1 for each category or level of the variable. 

We will work with the Python [patsy](https://patsy.readthedocs.io/en/latest/quickstart.html) package which creates **design matrices** from Pandas data frames. Patsy creates the design matrices using the, by now familiar, R-style modeling [formulas](https://patsy.readthedocs.io/en/latest/formulas.html). These design matrices can be used directly in statsmodels or scikit-learn models. 

> **Note:** In this lesson we will use the term design matrices. However, you often see the term **model matrices**. Don't be confused! These terms mean the same thing.

### Design matrix with an intercept term

To start our exploration of how model matrices are constructed we will start with an example using only numeric variables. We will use the [dmatrices](https://patsy.readthedocs.io/en/latest/API-reference.html) function from the patsy package, which returns both the feature and label matrices. Execute the code in the cell below to construct the label and design matrices for the model formula shown and examine the results. 

In [ ]:
print('Unique values of drive wheels = ' + str(auto_data.drive_wheels.unique()))
print('Unique values of body style = ' + str(auto_data.body_style.unique()))
formula = 'city_mpg ~ C(body_style)'
Y, X = dmatrices(formula, data=auto_data)
print('\nHead of label matrix \n{}\n'.format(Y[:5]))
print('Head design matrix \n{}'.format(X[:10]))

The label array, $Y$, looks much as you might expect. A one dimensional array of label values.  

The columns of the feature matrix might surprise you.  The first column is all 1's. This is the feature column for the **intercept term**. There are 4 containing dummy variables encoding the body style.

But notice that first two rows have only 0s for the dummy variables? How can this be? It turns out the intercept term is the mean of the first category of `body_style`. The other columns represent **contrasts** between the intercept and the **effect size** of the other categories. 

###  Design matrices without an intercept term 

We may not always want an intercept term. Patsy allows us to build a design matrix without an intercept term by including -1 in the formula. Execute the code in the cell below which does just this.     

In [ ]:
Y, X = dmatrices('city_mpg ~ -1 + C(body_style)', data=auto_data)
print('Head of label matrix \n{}\n'.format(Y[:10]))
print('Design matrix \n{}'.format(X[:10]))

The result is an array with 5 dummy variables, representing the 5 levels of the categorical variable. There is no longer an intercept column. 

Each of the levels of the categorical variable are now represented by a dummy variable. These dummy variables no represent the means for each level, not the contrasts. The feature matrix no longer has a column of 1s. By not using contrasts, the need for an intercept term is eliminated. 

We can test that this design matrix is **orthogonal**. By orthogonal, we mean that each dummy variable is not linearly dependent on any other. An orthogonal design matrix has several advantages. First, the inverse of the covariance matrix is guaranteed to exist. Second, we can directly interpret the model coefficients in terms of the response to the values of the predictor variables.  

The code in the cell below demonstrates the orthogonality of the design matrix by taking the dot products of every pairwise combination. Recall that the dot product between orthogonal vectors is zero. Execute this code and examine the results. 

In [ ]:
from itertools import combinations
for i,j in combinations(range(5), 2):
    print('For columns {0:1d} and {1:1d} the dot product = {2:3.2f}'.format(i,j,np.dot(X[:,i],X[:,j])))

## Design Matrices with Mixed Type Variables

It is also possible to create design matrices with both categorical variables and continuous numeric variables. The numeric variables must be normalized to be on the same scale as the dummy variables.      

The resulting design matrix, uses the means of the levels of a categorical variable as the model coefficients. In other words, these coefficients represent the value for each level rather than contrasts with an intercept.    

Execute the code in the cell below to create a design matrix with no intercept term with categorical variable `body_style` and normalized continuous variables `engine_size` and `curb_weight`, with no intercept. 

In [ ]:
Y, X = dmatrices('city_mpg ~ -1 + C(body_style) + curb_weight + engine_size', data=auto_data)
print('Head of label matrix \n{}\n'.format(Y[:5]))
print('Design matrix \n{}'.format(np.round(X[:5], 4)))

There are 7 columns in this design matrix. The first 5 columns are the dummy variables for the categorical variables for the contrasts of the levels. The last two columns are the values of the normalized numeric columns.   

One question you could ask, is if the columns of the design matrix are orthogonal? Execute the code in the cell below to find out.     

In [ ]:
for i,j in combinations(range(7), 2):
    print('For columns {0:1d} and {1:1d} the dot product = {2:3.2f}'.format(i,j,np.dot(X[:,i],X[:,j])))

Examine the results above. The dummy variables are orthogonal as they should be. However, the dummy variables are not orthogonal to the numeric variables. Further, the numeric columns are not orthogonal.          

> **Exercise 22-1:** To better understand how the encoding of the dummy variables as contrasts works do the following:       
> 1. Define and fit an ols model of `city_mpg` by `fuel_type + curb_weight + curb_weight^2` using the `ols` function from `statsmodels.formula.api`. Make sure you wrap the categorical variable in the `C()` operator and use the `I()` operator for the power term.     
> 2. Print the summary of this model.    

In [ ]:
## Code goes here




> Answer the following questions:   
> 1. Notice that there is 1 dummy variable representing 2 possible values. Which category of this variable is represented by the intercept term?     
> 2. Which of the parameters do you consider to be significant and why? 
> 3. What does the contrast variable tell you about the average city MPG for these categories and why?    
> 4. Based on adjusted $R^2$ and F-statistic does this model explain any of the variance of the city MPG?  

> **Answers:**      
> 1.         
> 2.     
> 3.      
> 4.       

>**Exercise 22-2:** You will now investigate how dropping the intercept term changes the behavior and interpretation of the model coefficients by doing the following:
     
> 1. Define and fit an ols model of `city_mpg` by `fuel_type + curb_weight + curb_weight^2` with no intercept using the `ols` function from `statsmodels.formula.api`. Name your model `city_mpg_ols_no_intercept`. Make sure you wrap the categorical variable in the `C()` operator.   
> 2. Print the summary of this model.    

In [ ]:
## Your code goes here




> 3. Next, run the code in the cell below to display diagnostic plots for the model.  

In [ ]:
def plot_resid_dist(resids):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    ## Plot a histogram
    sns.histplot(resids, bins=20, kde=True, ax=ax[0])
    ax[0].set_title('Histogram of residuals')
    ax[0].set_xlabel('Residual values')
    ## Plot the Q-Q Normal plot
    ss.probplot(resids, plot = ax[1])
    ax[1].set_title('Q-Q Normal plot of residuals')
    plt.show()
    
def residual_plot(df, predicted='predicted', resids='resids'):
    fig,ax = plt.subplots(figsize=(12,5))
    RMSE = np.std(df.loc[:,resids])
    sns.scatterplot(x=predicted, y=resids, data=df, ax=ax)
    ax.axhline(0.0, color='red', linewidth=1.0)
    ax.axhline(2.0*RMSE, color='red', linestyle='dashed', linewidth=1.0)
    ax.axhline(-2.0*RMSE, color='red', linestyle='dashed', linewidth=1.0)
    ax.set_title('PLot of residuals vs. predicted')
    ax.set_xlabel('Predicted values')
    ax.set_ylabel('Residuals')
    plt.show()    

auto_data.loc[:,'predicted'] = city_mpg_ols_no_intercept.predict(auto_data)
auto_data.loc[:,'resids'] = np.subtract(auto_data.loc[:,'city_mpg'], auto_data.loc[:,'predicted'])
ax = plot_auto()
sns.lineplot(x='curb_weight', y='predicted', hue='fuel_type', data=auto_data, ax=ax)

plot_resid_dist(auto_data.resids)
residual_plot(auto_data) 

fig,ax = plt.subplots(figsize=(8,8))
_=influence_plot(city_mpg_ols_no_intercept, ax=ax)

> 4. Notice there are some high leverage points associated with large residual values. Create and execute code to print the `city_mpg`, `fuel_type`, `curb_weight`, and `resids` columns for the 4 observations with the highest leverage in descending order of influence.   

In [ ]:
## Code goes here




> Examine the results and answer these questions:    
> 1. Notice that 2 of the model coefficients represent the mean for each category of fuel type and all coefficients are significant. How can you interpret the two values of the fuel coefficients?   
> 2. Compare the intercept term for the first model to the diesel fuel coefficient in the second model. Are these values and same and why? 
> 3. Have the slope coefficients for curb weight and curb weight squared changed between the models with and without an intercept term?   
> 4. How can you best describe the distribution of the residuals for this model?    
> 5. Examine the residual plot, noting that the pattern arising from the integer coded values of city MPG, but which can generally be ignored in evaluation of the residuals.  Are the residuals homoscedastic or heteroscedastic, and why?    
> 6. What statement can you make in terms of explaining the high leverage points?    

> **Answer:**      
> 1.         
> 2.     
> 3.     
> 4.     
> 5.       
> 6.     

At this point it is worth asking what the distributions of the parameters look like. To find out we can use a parametric bootstrap to get find an approximate distribution of the model parameters. To do so, execute the code in the cell below.     

In [ ]:
def resample_regression(df, n_boots, n_params=2, formula='y ~ x'):
    ## array to hold the bootstrap samples of the parameters
    boot_samples = np.zeros((n_boots,n_params))
    n_samples = df.shape[0]
    ## Loop over the number of resamples
    for i in range(n_boots):
        ## Create a bootstrap sample of the data frame
        boot_sample = df.sample(n=n_samples, replace=True)
        ## Compute the OLS model
        boot_model = smf.ols(formula=formula, data=boot_sample).fit()
        ## Save the model parameters in the array
        boot_samples[i,:] = boot_model._results.params
    return boot_samples

def compute_CI(values, parameter, p=0.05):   
    mean = np.mean(values)
    p = 100 * p / 2.0
    UCI = np.percentile(values, 100 - p)
    LCI = np.percentile(values, p)
    print('Parameter = ' + parameter)
    print(f'Mean = {mean}')
    print(f'Upper confidence interval = {UCI}')
    print(f'Lower confidence interval = {LCI}')
    return(mean, UCI, LCI)

def plot_boot_params(params, parameter='intercept'):
    mean, UCI, LCI = compute_CI(params, parameter)
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
    ## Plot a histogram
    ax[0].hist(params, bins=20)
    ax[0].axvline(mean, color='red', linewidth=1)
    ax[0].axvline(UCI, color='red', linewidth=1, linestyle='--')
    ax[0].axvline(LCI, color='red', linewidth=1, linestyle='--')
     
    ax[0].set_title('Histogram of parameter\n' + parameter)
    ax[0].set_xlabel('Model parameter values')
    ## Plot the Q-Q Normal plot
    ss.probplot(params, plot = ax[1])
    ax[1].set_title('Q-Q Normal plot of parameter\n' + parameter)
    plt.show()

param_boots = resample_regression(auto_data, 2000, n_params=4, formula=formula)
for i,parameter in enumerate(['diesel fuel', 'gas fuel','curb weight', 'curb weight squared']):  
    plot_boot_params(param_boots[:,i], parameter=parameter)

We can also get a feel for the variability of the model by bootstrap resampling the fit and examining the results. To do so, execute the code in the cell below and examine the results. This code computes and displays 100 samples of the bootstrapped regression fit line. There are two unusual aspects of this code. First, the line plot uses the `hue_order` argument to ensure that line hue corresponds to the correct program cases. Second, the bootstrap code in wrapped in a `with` statement to suppress printing the model convergence summary 100 times. Executing the resampling algorithm and displaying the lines with Seaborn may take some time.   

In [ ]:
def resample_regression_ols(df, formula,  ax, n_params=4, n_boots=100):
    ## array to hold the bootstrap samples of the parameters
    boot_samples = np.zeros((n_boots,n_params))
    n_samples = df.shape[0]
    ## Loop over the number of resamples
    for i in range(n_boots):
        ## Create a bootstrap sample of the data frame
        boot_sample = df.sample(n=n_samples, replace=True)
        boot_sample.index = range(len(boot_sample))

        ## Compute the OLS model
        boot_model = smf.ols(formula=formula, data=boot_sample).fit()    
        boot_sample.loc[:,'predicted'] = boot_model.predict(boot_sample)
        sns.lineplot(x='curb_weight', y='predicted', hue='fuel_type', hue_order=['gas','diesel'], data=boot_sample, ax=ax, alpha=0.1, legend=False);
        ## Save the model parameters in the array
        boot_samples[i,:] = boot_model._results.params
    return boot_samples

ax = plot_auto()
with contextlib.redirect_stdout(io.StringIO()):
    resample_regression_ols(auto_data, formula,  ax)

There are several observations one can make about these results:    
1. The distribution of the model parameters is generally close to Normal, with a few deviations.      
2. Cis are similar but not identical to the parametric estimates given in the model summary table. The difference being the nonparametric bootstrap estimates make no assumption about the distribution of the model parameters.   
3. The CIs are narrow for the fuel type coefficients, which makes sense since these values represent means that can be estimated with fairly low variance. The CIs for the slope coefficients are relatively wider, particularly for the squared terms, which is expected since slope is more sensitive to random variation in the sample.    
4. The variation in the bootstrap regression lines is reasonably small. As is often the case, the dispersion of the regression lines increases at the ends of the range of the independent variables.     

## Generalize Linear Model

Until now, we have been working strictly with linear regression models. Linear regression models have a numeric label. Further, ordinary linear regression assumes the label values are Normally distributed. 

But, what if the label has another distribution, particularly if the label is categorical? Using a more general form for the label values leads us to the **generalized linear model (GLM)**. Generalized linear model can be constructed for any response variable distribution in the exponential family. The exponential family includes many of the most commonly used probability distributions.    

GLMs are created by observing that for any distribution in the exponential we can define a **link function**. The link function maps a linear model space to the log of the (nonlinear) distribution space. The general form of the link function, $g()$, maps the expected value of the response variable, $y$, given the observation vector, $x$, to a linear model $\hat{\lambda} = \beta_0 + \beta_1\ x$:   

$$g \big( \mathtt{E}[y_i|x_i] \big)  = \hat{\lambda} = \beta_0 + \beta_1\ x$$

To find the value of the response variable we apply the inverse link function:    

$$\mathtt{E}[y_i|x_i]   = g^{-1} \big( \hat{\lambda} \big) = g^{-1} \big( \beta_0 + \beta_1\ x \big) $$

You may well wonder what the link function is for the Normal distribution, which is a member of the exponential family. In this case the link function and the inverse link functions are both unity or 1. This property leads to the ordinary least squares model with a Normally distributed response, we have already investigated.    

### Logistic Regression as a GLM

Now, we will look at a widely used variation of the generalized linear model using a **Binomial distribution**. This method is commonly known as [**logistic regression**](https://en.wikipedia.org/wiki/Logistic_regression). Logistic regression is widely used as a classification model. Logistic regression is linear model, with a binary response or label values, `{False, True}` or `{0, 1}`.  specifically, the response is computed as a log likelihood, leading to a Binomial distribution of the label values. 

The logistic regression algorithm is in the family of [**generalized liner models (GLM)**](https://en.wikipedia.org/wiki/Generalized_linear_model). GLMs use a transformation function to map a nonlinear function of probability to a linear model space, known as a **link function**. The **inverse link function** maps the linear model response to the nonlinear model space. 

To understand the binary response logistic regression as a GLM, we start with a model for the log-odds of response $1$ vs. $0$, given probability of success, or $y_i = 1$, $p_i$. For independent variable vector $x_i$, model parameter vector, $\mathbf{\beta}$, and binary response, $y_i = [0,1]$, we define the link function, know as the or **logit function**:   

$$logit(\mathtt{E}[y_i|x_i]) = logit(p(x_i)) = ln \Big(\frac{p(x_i)}{1-p(x_i)} \Big) =\beta_0 + \mathbf{\beta} 𝑥_i $$

The response of the linear model is transformed to the binomially distributed probability distribution through the **inverse link function**, know as the **inverse logit function**, or **logistic function**. After some algebra we can arrive at:   

\begin{align}
\lambda_i &= \beta_0 + \mathbf{\beta} 𝑥_i\\
p_i = \mathtt{E}[y_i|x_i] &= f(\lambda_i) = logit^{-1}(\lambda_i)  \\
p_i = f(x_i) &= logit^{-1}(\lambda_i) = \frac{1}{1 + e^{-\lambda_i}} = \frac{1}{1 + e^{-(\beta_0 + \mathbf{\beta} 𝑥_i)}}
\end{align}

We can gain an intuitive understanding of these transformations by creating a graph of the simple one-dimensional of the logistic function, or inverse logit function. The one-dimensional logistic function is parameterized by an center, $x_0$, and slope, $\kappa$.   

$$f(x)  = \frac{1}{1 + e^{-\kappa(x - x_0)}}$$

To simplify the example, we set $x_0 =0$ and $\kappa=1$. Execute the code in the cell below to compute and plot an example of the logistic function.

In [ ]:
# Plot the logistic transformation function (f(x) above)
x_seq = np.linspace(-7, 7, 100)

def log_fun(x, center=0, scale=1):
    e = np.exp(-scale*(x-center))
    log_out = 1./(1. + e)
    return log_out

log_fun_vectorized = np.vectorize(log_fun)

log_y = log_fun_vectorized(x_seq)

fig,ax = plt.subplots(figsize=(8,5))
ax.plot(x_seq, log_y)
ax.set_title('Standard Logistic Function')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.axvline(0, color='red', linewidth=2, linestyle='--')
ax.axhline(0.5, color='red', linewidth=2, linestyle='--')
ax.grid()

You can see that the logistic function crosses the threshold value of $0.5$ at $x=0$. A linear response greater than the threshold gives a model output of 1 and vice versa.        

Let's make this a bit more concrete with a simple example. Say we have a linear model with an intercept, $\beta_0$, and slope parameter, $\beta_1$.

$$\hat{\lambda} = \beta_0 + \beta_1\ x$$

Now, depending on the value of $\hat{y}$ we want to classify the output from a logistic regression model as either `0` or `1`. We can use the linear model in the logistic function as follows:

$$f(\hat{y}) = \frac{1}{1 + e^{-\lambda}} = \frac{1}{1 + e^{-(\beta_0 + \beta_1\ x)}} $$

In this way we transform the continuous output of the linear model defined on $-\infty \le \hat{y} \le \infty$ to a binary response, $0 \le f(\hat{y}) \le 1$ 

### Finding the maximum likelihood solution   

In Chapter 11 we investigated methods for finding maximum likelihood solutions at large scale. These methods are generally based on variations of the stochastic gradient descent (SGD) algorithms or quasi-Newton's methods like the limited memory Broyden–Fletcher–Goldfarb–Shanno (l-BFGS) algorithm. These algorithms are employed routinely to logistic regression problems on a massive scale.     

### What is Deviance?

OLS regression models are often evaluated based on variance ratios, such as the $R^2$ metric, or error metrics like RMSE. However given the nonlinear mapping between the linear model and the response, these methods are not suitable for generalized linear models.  

The significance of the GLM is expressed in terms of a statistic called **deviance**. It can be a bit of a challenge to wrap your head around what deviance really means. To further complicate the problem there are several commonly used forms of deviance. Here we will focus on one the simplest and easiest to understand forms, known as **null deviance**. Null deviance is 2 times the square of the log odds ratio between a model and a null model. Intuitively, the null model is simply informed guessing, and deviance is a measure of how much the model improves accuracy beyond guessing.  

The deviance statistic is $\chi^2$ distributed, and we can apply a significance test on a model.A model with small deviance is little better that informed guessing, and will have a small $\chi^2$ and not be considered a significant improvement. Whereas, a model with large deviance has a large $\chi^2$, demonstrating a significant improvement in accuracy.     

To understand binomial deviance, we can start with the expected value of the binomial log-likelihood function:  

$$\hat{l}(k, n | \pi) = log \binom{n}{k} + k\ log(\pi) + (n-k)\ log(1-\pi)$$

We can now construct the log likelihood of the null model, by the simple calculation of the binomial probability:  

$$p_\phi = \bar{y} = \frac{k}{n}$$

For the null model we set $\beta_0 = p_\phi$, so the logistic function for a null model is:    

$$f_{\phi}(\hat{y}) = \frac{1}{1 + e^{-p_{\phi}}} = \frac{1}{1 + e^{-\beta_0}}$$

And with expected log-likelihood:   

$$$\hat{l}_phi(k, n | p_{\phi}) = log \binom{n}{k} + k\ log(p_{\phi}) + (n-k)\ log(1-p_{\phi})$$

We can gain some intuitive understand of the behavior of the null model with an example. Consider the case where half the values of the response, $y$, are 1s. In this case, $p_\phi = n/2 = 0.5$. For each value of $x_i$, the model randomly selects a 1 or a 0 response with probability of 0.5. This model is random guessing and will have an accuracy of 0.5. In other words, the null model is no better in terms of predictive power than just saying that all values of $y$ are either 0 or all values are 1.    

Now that we understand the null model with can state the form of null deviance of a linear model, $logit(p(x_i)) = \beta_0 + \mathbf{\beta} 𝑥_i$, as the log of the square expected likelihood ratio:   

\begin{align}
D(\beta_0 + \mathbf{\beta} 𝑥_i) &= ln \Big( \frac{\hat{L}^2}{\hat{L}_{\phi}^2} \Big) \\
&= 2 \big( \hat{\mathcal{l}}  -  \hat{\mathcal{l}}_{\phi}  \big)
\end{align}

The expected null log-likelihood, $\hat{\mathcal{l}}_{\phi}$, is fixed by the observed response values. Therefore, the better the model, and higher the likelihood, $\hat{\mathcal{l}}$, the higher the deviance and therefore the value of $\chi^2$. Correspondingly, a model with large $\chi^2$ has greater significance and accuracy of predictions.    

In this section, we have focused on the logistic regression problem and binomial log-likelihood. The concepts discussed in this section can be applied to any GLM. In some cases, deviance can be used directly. In other cases log-likelihood ratio is used directly for model evaluation and significance.   

### Evaluation of binary responses

To evaluate models with binary output we can build a **confusion matrix** or **truth table** from counts. These counts are defined:

 - True Positives (TP): Count of cases that are positive and predicted to be positive.
 - True Negatives (TN): Count of cases that are negative and predicted to be negative.
 - False Positives (FP): Count of cases that are negative and predicted to be positive.
 - False Negatives (FN): Count of cases that are positive and predicted to be negative.

The other metrics are defined as follows:

- Accuracy = (TP + TN) / (TP + FP + TN + FN)
- Precision = TP / (TP + FP)
   - Precision is the fraction of the relevant class predictions are actually correct.
- Recall = TP / (TP + FN)
   - Recall is the fraction of the relevant class were we able to predict.

### Logistic Regression Example     

It's time to apply the foregoing theory to a practical problem. The goal for the analysis of the [HR dataset from Kaggle](https://www.kaggle.com/datasets/vjchoudhary7/hr-analytics-case-study) is to help a company identify which employees are most likely to leave. Apparently, these data were generated based on a simulation based on real-world cases. 

> **Exercise 22-3:** To create and evaluate a logistic regression model for the HR data, do the following:      
> 1. Execute the code in the cell below to load the dataset and center some of the independent variables.     

In [ ]:
hr_data = pd.read_csv('../data/HR_comma_sep.csv')
for col in ['satisfaction_level','average_montly_hours','last_evaluation']:
    hr_data.loc[:,col] = hr_data.loc[:,col] - np.mean(hr_data.loc[:,col])
hr_data.head()

> 2. Define and fit a model for employees leaving using this formula; `'left ~ satisfaction_level + average_montly_hours + last_evaluation + C(salary) + C(promotion_last_5years)'`.   
> 3. Print a summary of the fitted model.    

In [ ]:
## Code goes here




> 4. The prediction from the model are **binomial probabilities**. Execute the code in the cell below to compute and display these probabilities. The code displays the first 5 rows of the prediction probability along with a histogram of the probabilities for all cases and a violin plot by leaving status.    

In [ ]:
hr_data.loc[:,'predicted_prob'] = hr_glm.predict(hr_data)
print('Probabilities of leaving:')
print(hr_data.loc[:5,['predicted_prob', 'left']])

fig,ax = plt.subplots(1, 2, figsize=(16,6))
ax[0].hist(hr_data.loc[:,'predicted_prob'], bins=20)
ax[0].set_xlim(-0.01,1.0)
sns.violinplot(x='left', y='predicted_prob', data=hr_data, ax=ax[1])
plt.show()

> 5. The final step is to set a threshold and convert these probabilities to binary output, $[0,1]$ or $[leave,stay]$. The threshold is selected by examining the above plots, and then a bit of experimentation (hyperparameter search) to find a reasonable trade-off between precision and recall. To do so and display model performance metrics, execute the code in the cell below.  

In [ ]:
threshold = 0.35
hr_data.loc[:,'predicted'] = np.where(hr_data.loc[:,'predicted_prob'] > threshold, 1, 0)
print('\n\nPrediction of leaving:')
print(hr_data.loc[:5,'predicted'])

print('\nConfusion Matrix')
Confusion_Matrix = metrics.confusion_matrix(hr_data.loc[:,'left'], hr_data.loc[:,'predicted'])
accuracy = metrics.accuracy_score(hr_data.loc[:,'left'], hr_data.loc[:,'predicted'])#(Confusion_Matrix[1,1] +  Confusion_Matrix[0,0])/np.sum(Confusion_Matrix)
precision = metrics.precision_score(hr_data.loc[:,'left'], hr_data.loc[:,'predicted'])
recall = metrics.recall_score(hr_data.loc[:,'left'], hr_data.loc[:,'predicted'])
Confusion_Matrix = pd.DataFrame(Confusion_Matrix, index=['True Stay', 'True Leave'], columns = ['Predicted Stay', 'Predicted Leaving'])
print(Confusion_Matrix)
print(f"\nAccuracy = {round(accuracy, 3)}")
print(f"Precision = {round(precision, 3)}")
print(f"Recall = {round(recall, 3)}")

> Examine the results and answer these questions:    
> 1. Keeping in mind that a lower linear response corresponds to a lower probability of leaving the company, examine the model coefficients. Do the signs of these coefficient make sense and why?       
> 2. Examine the histogram of the probabilities. Keeping in mind that most employees do not leave the company, describe key properties of this distribution and how they relate to predicting leaving.               
> 3. Examine the confusion matrix (truth table) and the performance metrics. Do these metrics indicate the model effectively predicts people leaving the company and if not why not?          

> **Answers:**   
> 1.      
> 2.     
> 3.     

## GLM Example; Poisson Regression    

We will now apply a GLM to a Poisson regression example. Poisson regression is another example of a nonlinear response model. Along with some other distributions, Poisson regression is supported in many software packages, including [StatsModels Generalied Linear Models (GLM)](https://www.statsmodels.org/stable/glm.html) and Scikit-Learn [Generalized-linear-regression](https://scikit-learn.org/stable/modules/linear_model.html#generalized-linear-regression). Examination of this documentation shows a number of built in link functions and inverse link functions. Creating link functions for other distributions is supported as well.   

Recalling that the Poisson distribution has an exponential form with a single parameter, $\lambda$. The parameter, $\lambda$, is the expected arrival rate of the process. The predictions, $y$, given the observations, $x$ and the model parameter, $\lambda$, can then be written:      

$$log \big[ \mathtt{E}(y | x) \big] = x \lambda \Longleftrightarrow \mathtt{E}(y | x) = e^{x \lambda }$$

We can extend this relationship by using a linear model for $\lambda$, so that so that the expected arrival rate changes with the independent variables. For example, for a linear model with intercept, $\beta_0$, and $p$ dimensional slope parameter vector, $\vec{\beta}$. The Poisson regression estimate of $\lambda_i$, for a $p$ dimensional observation vector $\mathbf{x}_i $ is then:     

$$log(\lambda_i) = \beta_0 + \mathbf{x}_i \vec{\beta} \Longleftrightarrow \lambda_i = e^{(\beta_0 + \mathbf{x}_i $\vec{\beta})}$$

We can now use the above relations in the formulation of the Poisson distribution to find the link function ($e^{x_i \lambda_i }$) and inverse link function:   

\begin{align}
log \big[ \mathtt{E}(y_i |x_i) \big] = x_i \lambda_i &\Longleftrightarrow \mathtt{E}(y_ | x_i) = e^{x_i \lambda_i } \\
log \big[ \mathtt{E}(y_i |x_i) \big] = \beta_0 + \mathbf{x}_i \vec{\beta} &\Longleftrightarrow \mathtt{E}(y_ | x_i) = e^{\beta_0 + \mathbf{x}_i \vec{\beta} }
\end{align}

### Poisson regression example    

It is time to try and example. We will work with a dataset for the number (arrival in Poisson regression terms) of awards for high school students in three different academic programs. There are two independent variables, the program and the student's standardized math test score. These data are from the [UCLA Advanced Research Computing, Statistical Methods and Data Analytics](https://stats.oarc.ucla.edu/r/dae/poisson-regression/#:~:text=Examples%20of%20Poisson%20regression,the%20course%20of%2020%20years) group. Since the dependent variable is a count, number of awards, this problem is a candidate to try Poisson regression.     

Now, run th code in the cell below to load the dataset, prepare the values, and display the first few lines.   

In [ ]:
program_codes = {1:'General', 2:'Academic', 3:'Vocational'}
award_scores = pd.read_csv("https://stats.idre.ucla.edu/stat/data/poisson_sim.csv")
award_scores.loc[:,'prog'] = award_scores.loc[:,'prog'].replace(program_codes)
award_scores.head(10)

To get a feel from these data, execute the code in the cell below to examine the courts of awards by program type.  

In [ ]:
fig,ax = plt.subplots(figsize=(8,6))
sns.countplot(data=award_scores, x='num_awards',hue='prog', ax=ax);

As you can see, most students receive no awards, with a small number receiving up to 6.  

> **Exercise 22-4:** You will now construct and evaluate a Poisson regression model for these count data by these steps:    
> 1. Using [statsmodels.formula.api.poisson](https://www.statsmodels.org/dev/generated/statsmodels.formula.api.poisson.html) define and fit a model using the formula `'num_awards ~ math + C(prog)'`.   
> 2. Display the summary of the model.   

In [ ]:
## Code goes here




> 3. Execute the code in the cell below to display a plot of the data and the fitted regression lines.   

In [ ]:
award_scores['predicted'] = poisson_glm.predict(award_scores)

fig,ax = plt.subplots(figsize=(8,8))
sns.scatterplot(x='math', y='num_awards', hue='prog', data=award_scores, ax=ax);
sns.lineplot(x='math', y='predicted', hue='prog', data=award_scores, ax=ax, legend=False);
ax.set_title('Actual vs. predicted awards by program');
ax.set_xlabel('Math score');
ax.set_ylabel('Number of awards');

> 4. The next question is how variable are the parameter estimates? To find out, execute the code in the cell below to find the bootstrap distributions of the model parameters.     

In [ ]:
import contextlib
import io

def resample_regression_poisson(df, n_boots, n_params=2, formula='y ~ x'):
    ## array to hold the bootstrap samples of the parameters
    boot_samples = np.zeros((n_boots,n_params))
    n_samples = df.shape[0]
    ## Loop over the number of resamples
    for i in range(n_boots):
        ## Create a bootstrap sample of the data frame
        boot_sample = df.sample(n=n_samples, replace=True)
        ## Compute the OLS model
        boot_model = smf.poisson(formula=formula, data=boot_sample).fit()
        ## Save the model parameters in the array
        boot_samples[i,:] = boot_model._results.params
    return boot_samples


with contextlib.redirect_stdout(io.StringIO()):
    poisson_param_boots = resample_regression_poisson(award_scores, 2000, n_params=4, formula=formula)
for i,parameter in enumerate(['Intercept', 'General','Vocational', 'Math']):  
    plot_boot_params(poisson_param_boots[:,i], parameter=parameter)

> 5. Finally you can get a feel for the variability of the regression predictions by bootstrapping plotting the regression lines. Execute the code in the cell below and examine the results. The bootstrap resampling and display of the line plots with Seaborn will take some time.      

In [ ]:
def resample_regression_poisson(df, formula,  ax, n_params=4, n_boots=100):
    ## array to hold the bootstrap samples of the parameters
    boot_samples = np.zeros((n_boots,n_params))
    n_samples = df.shape[0]
    ## Loop over the number of resamples
    for i in range(n_boots):
        ## Create a bootstrap sample of the data frame
        boot_sample = df.sample(n=n_samples, replace=True)
        boot_sample.index = range(len(boot_sample))

        ## Compute the OLS model
        boot_model = smf.poisson(formula=formula, data=boot_sample).fit()    
        boot_sample.loc[:,'predicted'] = boot_model.predict(boot_sample)
        sns.lineplot(x='math', y='predicted', hue='prog', hue_order=['Vocational','General', 'Academic'], data=boot_sample, ax=ax, alpha=0.1, legend=False);
        ## Save the model parameters in the array
        boot_samples[i,:] = boot_model._results.params
    return boot_samples

fig,ax = plt.subplots(figsize=(8,8))
sns.scatterplot(x='math', y='num_awards', hue='prog', data=award_scores, ax=ax);
with contextlib.redirect_stdout(io.StringIO()):
    resample_regression_poisson(award_scores, formula,  ax)

> Examine the result and answer the following questions:     
> 1. Examine the distributions of the the model coefficients. How can you describe the distributions with respect to how close they are or not close to Normal?        
> 2. Next consider the bootstrap regression lines, keeping in mind that based on the regression model the value of the expected arrival rate parameter, $\lambda$ increases with the value of `math`, and that $\lambda$ is also the variance. How can you explain the increasing dispersion of the regression lines toward the right of the plot above?   
> 3. Given the dispersion of the regression lines, what are some reasonable inferences you can make and why?         

> **Answers:**    
> 1.     
> 2.     
> 3.    

#### Copyright 2017, 2018, 2019, 2020, 2021, 2022 Stephen F Elston. All rights reserved. 